In [ ]:
import getpass
import pprint
import os

import pandas
import qmenta.client
from tqdm import tqdm

# 1. Sign-up and login in QMENTA

Go to https://platform.qmenta.com/#/register, create an account to use the QMENTA platform, and then login with your recently created account to acces the platform.

#### PROMOTIONAL CODE: (INSERT CODE HERE)

![QMENTA Platform Registration form](assets/qmenta_platform_registration.png)

Check this **Getting started** article in case you need help with registration and login

https://support.qmenta.com/hc/en-us

# 2. Set up the QMENTA Client

Fill in the following cells with your username and password

In [ ]:
username = 'username'

In [ ]:
password = getpass.getpass('Password [{}]:'.format(username))

Instantiate a QMENTA Client account, and get a Project instance to interact with the Hackaton project in the platform 

In [ ]:
acc = qmenta.client.Account(username=username, password=password)

In [ ]:
project = acc.get_project('CNIC-QMENTA 1000 Brains Challenge')

# 3. List all the subjects in the project and their metadata

Get all the subjects and sort them by patient_secret_name

In [ ]:
subjects_metadata = project.get_subjects_metadata()  # This returns a list of dictionaries
subjects_metadata = sorted(subjects_metadata, key=lambda x: x['patient_secret_name'])

We should have 1100 subjects

In [ ]:
assert len(subjects_metadata) == 1100

Let's have a look at the data structure that represents each subject

In [ ]:
subjects_metadata[0]

The relevant fields are:
- **patient_secret_name**: unique identifier for this subject in this project. We also refer to this field as **SubjectID** 
- **md_set**: identifies if the subject belongs to the 'train' or the 'test' set
- **md_age**: the age of the subject

Let's separate the subjects between train and set

In [ ]:
train_subjects = [x for x in subjects_metadata if x['md_set'] == 'train']
assert len(train_subjects) == 1000

In [ ]:
test_subjects = [x for x in subjects_metadata if x['md_set'] == 'test']
assert len(test_subjects) == 100

We will only use the subjects in 'train' to build a CSV which maps the SubjectID with the Age.

In [ ]:
train_csv_info_dict = {x['patient_secret_name']: {'Age': x['md_age']} for x in train_subjects}

In [ ]:
train_csv_dataframe = pandas.DataFrame.from_dict(train_csv_info_dict, orient='index')

In [ ]:
train_csv_dataframe

Let's create a local folder to store all the data from the Hackaton 

In [ ]:
hackaton_dir = os.path.expanduser('~/qmenta_cnic_1000_brains_challenge')
print(hackaton_dir)

In [ ]:
os.makedirs(hackaton_dir, exist_ok=True)

Now we will store the pandas DataFrame as a CSV in this folder, so that we can use it later to train a Machine Learning model.

In [ ]:
train_csv_dataframe.to_csv(os.path.join(hackaton_dir, 'train.csv'))

# 4. Fetch the analysis results for all subjects

We will again list all the completed analysis in the platform and sort them by patient secret name

In [ ]:
analysis = project.list_analysis()
analysis = [x for x in analysis if x['state'] == 'completed']
analysis = sorted(analysis, key=lambda x: x['patient_secret_name'])

Again we should have 1100 analyses

In [ ]:
assert len(analysis) == 1100

Let's see the data structure that represents each analysis

In [ ]:
analysis[0]

As mentioned before, the analysis run for this database quantifies volumetry and morphometry of the brain given a structural MR image, concretely a T1-weighted MR image.
The specific tool that we used is build upon [ANTs](The specific tool that we used is build upon [ANTs]). You can learn about the specifics of this tool in this support article [here](https://support.qmenta.com/hc/en-us/articles/115000760611-ANTs-Morphology-2-1-0-).

The important field in this case is the **out_container_id**, because this identifies the data container that stores the result files. We will need to download one or more of these result files to use them as the predictor variables for our ML model.

Let's see the typical set of files produced by an ANTs analysis

In [ ]:
out_container_id_example = analysis[0]['out_container_id']
results_files_example = project.list_container_files_metadata(out_container_id_example)
pprint.pprint(results_files_example)

As you can see ANTs generates a lot of intermediate results, however we are only interested in few things:
- T1_strip.nii.gz (modality=T1, tags=strip): skull-stripped brain, in case you want to train a Machine or Deep Learning algorithm on the raw structural data.
- thickness.nii.gz (tags=thickness): thickness map of the brain, in which each voxel belonging to the cortex has a value that indicates the thickness in mm.
- tissueSegmentation.nii.gz (tags=tissue_segmentation): segmentation map of the brain into its main tissues, namely Gray Matter, White Matter, CerebroSpinal Fluid (CSF), Deep Brain, Brain-Stem and Cerebellum.
- labels.nii.gz (tags=labels): Brain parcellation of the cortex and other important structures.
- volumetric.csv: CSV with volume, average thickness and standard deviation of thickness information for each tissue and region in the brain.

In our example we will only use the information provided by **volumetric.csv**, however you are free to use any of the result files available in there, and even the original T1 image.

Let's download the volumetric.csv for our example analysis and inspect it using pandas

In [ ]:
project.download_file(container_id=out_container_id_example, file_name='volumetric.csv', local_filename='/tmp/volumetric.csv', overwrite=True)

In [ ]:
volumetric_csv_example = pandas.read_csv('/tmp/volumetric.csv')

In [ ]:
volumetric_csv_example

Now we can download all the **volumetric.csv** for all subjects in order to use this information to train an ML algorithm to predict the age.

First we will create a folder to store all the volumetric data in our computer from the train and test sets

In [ ]:
train_volumetric_data_dir = os.path.join(hackaton_dir, 'train')
print(train_volumetric_data_dir)

In [ ]:
os.makedirs(train_volumetric_data_dir, exist_ok=True)

In [ ]:
test_volumetric_data_dir = os.path.join(hackaton_dir, 'test')
print(test_volumetric_data_dir)

In [ ]:
os.makedirs(test_volumetric_data_dir, exist_ok=True)

We distinguish between analysis from the train and test set

In [ ]:
train_subjects_set = set([x['patient_secret_name'] for x in train_subjects])
test_subjects_set = set([x['patient_secret_name'] for x in test_subjects])

In [ ]:
train_analysis = [x for x in analysis if x['patient_secret_name'] in train_subjects_set]
test_analysis = [x for x in analysis if x['patient_secret_name'] in test_subjects_set]

For each analysis we download the volumetric data

In [ ]:
for analysis_instance in tqdm(train_analysis):
    analysis_container_id = analysis_instance['out_container_id']
    patient_name = analysis_instance['patient_secret_name']
    volumetric_filepath = os.path.join(train_volumetric_data_dir, '{}_volumetric.csv'.format(patient_name))
    project.download_file(container_id=analysis_container_id, file_name='volumetric.csv', local_filename=volumetric_filepath, overwrite=True)

In [ ]:
for analysis_instance in tqdm(test_analysis):
    analysis_container_id = analysis_instance['out_container_id']
    patient_name = analysis_instance['patient_secret_name']
    volumetric_filepath = os.path.join(test_volumetric_data_dir, '{}_volumetric.csv'.format(patient_name))
    project.download_file(container_id=analysis_container_id, file_name='volumetric.csv', local_filename=volumetric_filepath, overwrite=True)